In [ ]:
%matplotlib inline
import os
import numpy as np
import json
filepath='D:/measuring'
import scipy.signal
c = 2.997e8
os.chdir(filepath)
from analysis.lib.fitting import fit, common; reload(common)
from analysis.lib.tools import plot
import matplotlib.pyplot as plt


In [ ]:
import analysis.scripts.cavity.oscilloscope_analysis_linewidth as oscillo_ana

In [ ]:
LWs_LT = [2.04,2.05,1.83,1.96,2.01,2.0,1.78,1.8,1.96,1.61,1.86,1.36,1.64,2.27,2.22,2.26,1.87,2.05,1.99,1.94]
nr_lws = len(LWs_LT)
avg_lw = np.average(LWs_LT)
u_avg_lw = np.std(LWs_LT)

print '%d fitted; average linewidth = %.2f +- %.2f'%(nr_lws,avg_lw,u_avg_lw)

In [ ]:
lws = np.array([])

In [ ]:
indir = 'D:\measuring/data/20160907/145700_reflection_scope_trace'
filename = 'REFL000'
EOM_freq= 6.
reload(oscillo_ana)

oa_all = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
oa_all.get_data(use_timetrace=False,x_min=None,x_max=None,data_col=3)
oa_all.plot_data(plot_mod=True,figsize=(16,4),fmt='o')

x_min=64000
x_max=x_min+5000
oa_single = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
oa_single.get_data(use_timetrace=False,x_min=x_min,x_max=x_max,data_col=3)#1400)
oa_single.plot_data(plot_mod=False,figsize=(16,4),fmt='o')


In [ ]:


# for j,filename in enumerate(filenames):
#     oa = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
#     oa.get_data(use_timetrace=False,x_min=0,x_max=10000,data_col=3)#1400)
#     oa.plot_data()#plot_mod=True)

#     # lws_old=lws[:-1]
#     lws = np.array([])#lws_old## 
#     As = np.array([])#lws_old## 
#     gammas = np.array([])#lws_old## 
#     nr_lws=10

#     x0 =x0s[j]
#     windowsize=5000# print len(x)
#     # xi = int(len(x)/nr_lws)
#     # print xi 

#     x,y = oa.get_data(use_timetrace=False,data_col=3)
#     # lw,u_lw = oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq)
#     # print lw

# 

In [ ]:
print len(good_lws)
print good_lws[-1]
print good_u_lws
print good_chisqs
print 'average lw %.2f +- %.2f'%(np.average(good_lws[:-1]) ,np.std(good_lws[:-1]))



In [ ]:
lw,u_lw,A1,u_a1,g,u_g,chi_sq = oa_single.plot_and_fit_with_EOM(EOM_freq=EOM_freq,
                                g_a1 = 0.01,g_A1 = 120000,g_A2=20000,g_A3=20000,g_gamma1=200,g_dx = 500,
                                show_fit=True,print_fit=True,plot_name=str(x_min))

print chi_sq

In [ ]:
lws = [1.18,1.48,1.37,2.22]

In [ ]:
#functions for analysis of linewidths from several files, 
#taking into account the up down modulation

def get_up_down_lws(fitted_lws,first_lw_direction):
    even_lws=fitted_lws[::2]
    odd_lws = fitted_lws[1::2]
    if len(even_lws)>len(odd_lws):
        even_lws=even_lws[:-1]
    updown_mean_lws=np.add(even_lws,odd_lws)/2        
    if first_lw_direction=='u':
        lws_up = even_lws
        lws_down  = odd_lws
    elif first_lw_direction=='d':
        lws_up = odd_lws
        lws_down  = even_lws
    else:
        print 'specify a valid modulation direction for the first linewidth'
        return
    return updown_mean_lws,lws_up,lws_down

def add_to_analysis_dict(analysis,lws_up,lws_down,updown_mean_lws):
    analysis['lws_up']=lws_up
    analysis['lws_down']=lws_down
    analysis['updown_mean_lws']=updown_mean_lws
    analysis['avg_up']=np.average(lws_up)
    analysis['u_avg_up']=np.std(lws_up)
    analysis['avg_down']=np.average(lws_down)
    analysis['u_avg_down']=np.std(lws_down)
    analysis['avg_updown_mean']=np.average(updown_mean_lws)
    analysis['u_avg_updown_mean']=np.std(updown_mean_lws)
    return analysis

def print_results(analysis):
    print 'avg_up', analysis['avg_up'],analysis['u_avg_up']
    print 'avg_down', analysis['avg_down'],analysis['u_avg_down']
    print 'avg_updown_mean', analysis['avg_updown_mean'],analysis['u_avg_updown_mean']

#functions for saving in json
def default(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError('Not serializable')

def save_to_json_file(indir,filename,analysis_dict):
    f = open(os.path.join(indir,filename+'.json'), 'w')
    json.dump(analysis_dict,f,default=default)
    f.close()

def save_to_txt_file(indir,filename,nparray):
    f = open(os.path.join(indir,filename+'.txt'), 'w')
    np.savetxt(f,nparray)
    f.close()
  

In [ ]:

# Parameters for

#g_x02 = 0.02
# g_gamma2 = 0.002

nr_datafiles = 9

linewidths = np.zeros((nr_datafiles,2))

reload(oscillo_ana)

for i in np.arange(nr_datafiles):
    fixed =[]#[0,1,2,5]
    
    g_a1 = 0.002
    g_A1 = 0.00004
    g_x01 = 0.002
    g_gamma1 = 0.0004
    g_dx = 0.0015
    g_A2 = 0.00002
    
    if i == 2:
        g_gamma1 = 0.0035
    indir='D:\measuring\data/20160430\OFF_diamond'#'D:\measuring\data/20160430\OFF_diamond'
    filename='OFFD00'+str(i)
    linewidth, u_linewidth = oscillo_ana.plot_and_fit(indir, filename,
                                                      g_a1, g_A1, g_x01, g_gamma1, g_dx, g_A2,fixed=fixed)
    print linewidth, u_linewidth
    linewidths[i,0]= linewidth
    linewidths[i,1] = u_linewidth
print linewidths

f = open(os.path.join(indir,'linewidth_data.txt'), 'w')
np.savetxt(f,linewidths)
f.close()

In [ ]:
nu=470.e12
Q = nu/(linewidths[1])
print Q
nu_FSR=c/(2*cavity_lengths[1])
F=Q*nu_FSR/nu
print F
F=nu_FSR/linewidths[1]
print F
print nu_FSR


In [ ]:
def get_avg_values_from_json(V,Lnr,day_dir,tag=''):
    V_str = '%1.3f'%V
    string_name = 'V'+str(V)
    #string_name = '%sL%d_V%sp%s'%(tag,Lnr,V_str[0],V_str[2:])
    print string_name
    datapath = os.path.join(day_dir,string_name)
    analysis=0
    
    print datapath
    for filename in os.listdir(datapath):
        if filename.endswith("analysis.json"):
            print filename 
            f = open(os.path.join(datapath,filename), 'r')
            analysis = json.load(f)
            f.close()
            
    if analysis==0:
        print 'analysis file not found!'
        return datapath, 0
        
            
    avg_values = np.array([[analysis['avg_up'], analysis['u_avg_up'],\
        analysis['avg_down'],analysis['u_avg_down'],\
        analysis['avg_updown_mean'],analysis['u_avg_updown_mean']]])
    return datapath,avg_values


def get_avg_array_for_Vs(Vs,Lnr,day_dir,tag=''):
    for i,V in enumerate(Vs):
        datapath,avg_values=get_avg_values_from_json(V,Lnr,day_dir,tag=tag)
        if i==0:
            datapath,avg_array=get_avg_values_from_json(V,Lnr,day_dir,tag=tag)
        else:
            avg_array=np.append(avg_array,avg_values,axis=0)
    return datapath,avg_array

def get_LT_avg_values_from_json(V,L_dir,nr_files=2):
    string_name = 'V'+str(V)
    print string_name
    datapath = os.path.join(L_dir,string_name)
    analysis=0
    
    print datapath
    lws=np.array([])
    for filename in os.listdir(datapath):
        if filename.endswith("analysis.json"):
            print filename 
            f = open(os.path.join(datapath,filename), 'r')
            analysis = json.load(f)
            f.close()
            lws = np.append(lws,analysis['lws'])
    return datapath,np.average(lws),np.std(lws)

def get_LT_avg_array_from_json(Vs,L_dir):
    for i,V in enumerate(Vs):
        datapath,avg_values,std_values=get_LT_avg_values_from_json(V,L_dir)
        if i==0:
            datapath,avg_array,std_array=get_LT_avg_values_from_json(V,L_dir)
        else:
            avg_array=np.append(avg_array,avg_values)
            std_array=np.append(std_array,std_values)
    return datapath,avg_array,std_array
  

In [ ]:
def plot_updown_lws(avgs,Ns,ax=None):
    if ax==None:
        fig,ax = plt.subplots()
    ax.errorbar(Ns,avgs[:,0],yerr=avgs[:,1],fmt='ro',label='upgoing slope')
    ax.errorbar(Ns,avgs[:,2],yerr=avgs[:,3],fmt='go',label='downgoing slope')
    ax.errorbar(Ns,avgs[:,4],yerr=avgs[:,5],fmt='bo',label='up and down mean')
    ax.set_xlabel('cavity length (lambda/2)')
    ax.set_ylabel('linewidth (GHz)')
    ax.legend()
    return(ax)

def plot_mean_lws_from_avgs(avgs,Ns,ax=None,style='bo',label='mean'):
    plot_mean_lws(avgs[:,4],avgs[:,5],Ns,ax=None,style='bo',label='mean')

def plot_mean_lws(avg_lws,u_avg_lws,Ns,ax=None,style='bo',label='mean'):
    if ax==None:
        fig,ax = plt.subplots()
    ax.errorbar(Ns,avg_lws,yerr=u_avg_lws,fmt=style,label=label)
    ax.set_xlabel('cavity length (lambda/2)')
    ax.set_ylabel('linewidth (GHz)')
    ax.legend()
    return(ax)

def plot_finesse_vs_length_from_avgs(avgs,Ns,lambda_c=636.6e-9):
    ax = plot_finesse_vs_length(avgs[:,4],avgs[:,5],Ns,lambda_c=lambda_c)

def plot_finesse_vs_length(avg_lws,u_avg_lws,Ns,lambda_c=636.6e-9):
    Finesses = c/(np.multiply(Ns,lambda_c))/np.multiply(avg_lws,1.e9)
    u_Finesses = c/(np.multiply(Ns,lambda_c))/np.multiply((avg_lws**2),1.e9)*u_avg_lws
    fig,ax = plt.subplots()
    print Finesses,u_Finesses
    ax.errorbar(Ns,Finesses,yerr=u_Finesses,fmt='bo',label='mean Finesse')
    ax.set_xlabel('cavity length (lambda/2)')
    ax.set_ylabel('Finesse')
    ax.legend()
    return(ax)

def fit_lws(linewidths,u_linewidths,Ns,g_Finesse,datapath,ax=None,lambda_c=636.6e-9):
    #cavity_lengths =  Ns#636.6e-9*Ns/2 # in um
    g_a = 0#0.1*1.e9
    g_b = (c)/(2*g_Finesse)*1.e-9*2/lambda_c #*1.e9
    print g_b
    print Ns
    print linewidths
    p0, fitfunc, fitfunc_str = common.fit_inverse(g_a, g_b)
    fit_result = fit.fit1d(Ns,linewidths, None, 
                           p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=[0])
    if ax == None:
        fig,ax = plt.subplots(figsize=(8,4))
        ax.errorbar(Ns,linewidths,yerr=u_linewidths,fmt = 'bo')
    x = np.linspace(min(Ns),max(Ns),10*len(Ns))
    plot.plot_fit1d(fit_result,x, ax=ax, label='Fit',show_guess=True, 
                    plot_data=False,color='red', print_info= False)
    b_fit=fit_result['params_dict']['b']#cav length in um, linewidth in GHz
    u_b_fit = fit_result['error_dict']['b']
    print b_fit
    Finesse = c/(2*b_fit)*1.e-9*2/lambda_c
    u_Finesse = c/(2*b_fit**2)*1.e-9*2/lambda_c*u_b_fit
    ax.set_title(datapath)
    ax.legend(title='Finesse = %d $\pm$ %d'%(Finesse,u_Finesse))
    plt.savefig(os.path.join(datapath,'plot_lw_vs_L.png'))
    plt.show()
    plt.close()
    return Finesse

In [ ]:
#LT ON DIAMOND, POS 4
dir_L2 = 'D:\measuring\data/20160909/LT_OND_POS4_L2'
dir_L3 = 'D:\measuring\data/20160909/LT_OND_POS4_L3'
dir_L4 = 'D:\measuring\data/20160909/LT_OND_POS4_L4'
dir_L5 = 'D:\measuring\data/20160909/LT_OND_POS4_L5'
N2s= [42,43,44]
N3s = [44,45,46,47]
N4s = [45,46,47,48]
N5s=[48,49,50,51]#educated guess for now
V2s=[0,1,2]
V3s=[0,1,2,3]
V4s=[0,1,2,3]
V5s = [0,1,2,3]
datapath,avg_array2,std_array2 = get_LT_avg_array_from_json(V2s ,dir_L2)
datapath,avg_array3,std_array3 = get_LT_avg_array_from_json(V3s ,dir_L3)
datapath,avg_array4,std_array4 = get_LT_avg_array_from_json(V4s ,dir_L4)
datapath,avg_array5,std_array5 = get_LT_avg_array_from_json(V5s ,dir_L5)


avg_array =np.append(np.append(np.append(avg_array2,avg_array3),avg_array4),avg_array5)
std_array =np.append(np.append(np.append(std_array2,std_array3),std_array4),std_array5)
Ns = np.append(np.append(np.append(N2s,N3s),N4s),N5s)


ax = plot_finesse_vs_length(avg_array,std_array,Ns)

plot_mean_lws(avg_array,std_array,Ns)

In [ ]:
#LT ON DIAMOND , POS 2
dir_L5 = 'D:\measuring\data/20160905/LT_OND_POS2_L5'
dir_L6 = 'D:\measuring\data/20160905/LT_OND_POS2_L6'
dir_L7 = 'D:\measuring\data/20160905/LT_OND_POS2_L7'

N2s= [69]
avg_array2 = [1.92]
std_array2 = [0.22]
V5s= [0,1,2,3]
N5s = [53,54,55,56]
V6s=[0,1,2,3]
N6s=[48,49,50,51]
V7s = [0,1,2,3]
N7s=[44,45,46,47]


datapath,avg_array5,std_array5 = get_LT_avg_array_from_json(V5s ,dir_L5)
datapath,avg_array6,std_array6 = get_LT_avg_array_from_json(V6s ,dir_L6)
datapath,avg_array7,std_array7 = get_LT_avg_array_from_json(V7s ,dir_L7)


avg_array = np.append(np.append(np.append(avg_array5,avg_array6),avg_array7),avg_array2)
std_array = np.append(np.append(np.append(std_array5,std_array6),std_array7),std_array2)
Ns = np.append(np.append(np.append(N5s,N6s),N7s),N2s)

print avg_array

ax = plot_finesse_vs_length(avg_array,std_array,Ns)

plot_mean_lws(avg_array,std_array,Ns)

In [ ]:
#LT OFF DIAMOND , POS 3
dir_L0 = 'D:\measuring\data/20160907/LT_POS3_OFFD_L0'


N0s= [17]
V0s = [0]

datapath,avg_array0,std_array0 = get_LT_avg_array_from_json(V0s ,dir_L0)

avg_array = avg_array0
std_array = std_array0
Ns = N0s

print avg_array

ax = plot_finesse_vs_length(avg_array,std_array,Ns)

plot_mean_lws(avg_array,std_array,Ns)

In [ ]:
# ON DIAMOND LINEWIDTH< LENGHTS< POS2
dir_L2 = 'D:\measuring\data/20160825/OND_POS2L2'
dir_L3 = 'D:\measuring\data/20160825/OND_POS2L3'
dir_L4 = 'D:\measuring\data/20160825/OND_POS2L4'
dir_L5 = 'D:\measuring\data/20160825/OND_POS2L5'
dir_L6 = 'D:\measuring\data/20160825/OND_POS2L6'
dir_L7 = 'D:\measuring\data/20160825/OND_POS2L7'
dir_L8 = 'D:\measuring\data/20160825/OND_POS2L8'

L2_N0 = 70#73
L3_N0 = 65#68
L4_N0 = 59#62
L5_N0 = 54#57
L6_N0 = 49#48
L7_N0 = 45#44
L8_N0 = 44#42

nr_files_L3= 9
V2s=np.arange(0,0+9,1)
V3s=np.arange(0,0+9,1)
V4s=np.arange(0,0+9,1)
V5s=np.arange(0,0+9,1)
V6s=np.arange(0,0+9,1)
V7s=np.arange(0,0+9,1)
V8s=np.arange(0,0+7,1)



print V2s

# V3s=np.array([1.373,2.859,4.062,5.152,6.218,7.224,8.198,9.268])
# V4s=np.array([0.951,2.390,3.646,4.783,5.843,6.934,7.975,8.977])
# V5s=np.array([1.359,3.383,4.565,5.620,6.695,7.727,8.734,9.832])

N2s = np.arange(L2_N0 ,L2_N0+9,1)#55,63
N3s= np.arange(L3_N0 ,L3_N0+9,1)
N4s= np.arange(L4_N0 ,L4_N0+9,1)
N5s= np.arange(L5_N0 ,L5_N0+9,1)
N6s= np.arange(L6_N0 ,L6_N0+9,1)
N7s= np.arange(L7_N0 ,L7_N0+9,1)
N8s= np.arange(L8_N0 ,L8_N0+7,1)
# N3s = np.arange(47,55,1)#49,57
# N4s = np.arange(44,52,1)
# N5s = np.arange(39,47,1)
# print V3s
# print N0s
# print N3s
# print N4s
# 
datapath,avgs2=get_avg_array_for_Vs(V2s,1,dir_L2,tag='')
datapath,avgs3=get_avg_array_for_Vs(V3s,1,dir_L3,tag='')
datapath,avgs4=get_avg_array_for_Vs(V4s,1,dir_L4,tag='')
datapath,avgs5=get_avg_array_for_Vs(V5s,1,dir_L5,tag='')
datapath,avgs6=get_avg_array_for_Vs(V6s,1,dir_L6,tag='')
datapath,avgs7=get_avg_array_for_Vs(V7s,1,dir_L7,tag='')
datapath,avgs8=get_avg_array_for_Vs(V8s,1,dir_L8,tag='')

# datapath,avgs3=get_avg_array_for_Vs(V3s,3,day_dir_20160822,tag='OND_POS2_')
# datapath,avgs4=get_avg_array_for_Vs(V4s,4,day_dir_20160822,tag='OND_POS2_')
# datapath,avgs5=get_avg_array_for_Vs(V5s,5,day_dir_20160822,tag='OND_POS2_')

# avgs = avgs3
# Ns = N3s
# avgs = np.append(avgs2,avgs3,axis=0)
# Ns = np.append(N2s,N3s)
# avgs = np.append(np.append(avgs2,avgs3,axis=0),avgs4,axis=0)
# Ns = np.append(np.append(N2s,N3s),N4s)
# avgs = np.append(np.append(np.append(avgs2,avgs3,axis=0),avgs4,axis=0),avgs5,axis=0)
# Ns = np.append(np.append(np.append(N2s,N3s),N4s),N5s)
# avgs = np.append(np.append(np.append(np.append(avgs2,avgs3,axis=0),avgs4,axis=0),avgs5,axis=0),avgs6,axis=0)
# Ns = np.append(np.append(np.append(np.append(N2s,N3s),N4s),N5s),N6s)
# avgs = np.append(np.append(np.append(np.append(np.append(avgs2,avgs3,axis=0),avgs4,axis=0),avgs5,axis=0),avgs6,axis=0),avgs7,axis=0)
# Ns = np.append(np.append(np.append(np.append(np.append(N2s,N3s),N4s),N5s),N6s),N7s)
avgs = np.append(np.append(np.append(np.append(np.append(np.append(avgs2,avgs3,axis=0),avgs4,axis=0),avgs5,axis=0),avgs6,axis=0),avgs7,axis=0),avgs8,axis=0)
Ns = np.append(np.append(np.append(np.append(np.append(np.append(N2s,N3s),N4s),N5s),N6s),N7s),N8s)

avgs_zoom= np.append(np.append(avgs7,avgs8,axis=0),avgs6,axis=0)
Ns_zoom= np.append(np.append(N7s,N8s,axis=0),N6s,axis=0)

# avgs=np.append(np.append(np.append(avgs0,avgs3,axis=0),avgs4,axis=0),avgs5,axis=0)
# Ns=np.append(np.append(np.append(N0s,N3s),N4s),N5s)
# avgs = np.append(np.append(avgs0,avgs1,axis=0),avgs2,axis=0)
# Ns = np.append(np.append(N0s,N1s),N2s)

ax = plot_updown_lws(avgs,Ns)
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L.png'))
plt.show()

ax = plot_updown_lws(avgs_zoom,Ns_zoom)
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L_zoom.png'))
plt.show()

ax = plot_finesse_vs_length(avgs,Ns)
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'Finesse_vs_L.png'))
plt.show()


ax = plot_mean_lws(avgs,Ns, label='mean pos 1')
ax = plot_mean_lws(avgspos0,Nspos0, ax=ax, style ='go',label='mean pos 0')
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L_2positions.png'))
plt.show()


In [ ]:
# off diamond values

day_dir_20160818 = 'D:\measuring\data/20160818'
V0s = np.array([1.129,2.473,3.482])
V2s = np.array([6.083,7.258,8.269,9.246])
day_dir_20160819 = 'D:\measuring\data/20160819'
V3s = np.array([1.402,2.646,3.781,4.846,5.864,6.890])
N0s = np.array([21,22,23])#possibly the N0s should be 21,22,23->not 100%clear. -> leave out?
N2s = np.array([13,14,15,16])
N3s = np.array([17,18,19,20,21,22])

datapath,avgs0=get_avg_array_for_Vs(V0s,0,day_dir_20160818)
datapath,avgs2=get_avg_array_for_Vs(V2s,2,day_dir_20160818)
datapath,avgs3=get_avg_array_for_Vs(V3s,3,day_dir_20160819)

avgs = np.append(np.append(avgs0,avgs2,axis=0),avgs3,axis=0)
Ns = np.append(np.append(N0s,N2s),N3s)

linewidths = avgs[:,4]
u_linewidths = avgs[:,5]
ax = plot_updown_lws(avgs,Ns)
Finesse=fit_lws(linewidths,u_linewidths,Ns,g_Finesse=30000,datapath=datapath, ax=ax)
plot_finesse_vs_length(avgs,Ns)
print Finesse



In [ ]:
# ON DIAMOND LINEWIDTH< LENGHTS< POS0
day_dir_20160819 = 'D:\measuring\data/20160819'
V0s=np.array([2.935,4.088,5.183,6.182])
V1s=np.array([2.139,3.587,4.753,5.836,6.891,7.928,8.909,9.958])
V2s=np.array([6.957,8.738,9.866])

N0s = np.array([54,55,56,57])
N1s = np.array([45,46,47,48,49,50,51,52])#np.array([44,45,46,47,48,49,50,51])
N2s = np.array([52,53,54])#np.array([51,52,53])
print N0s
print N1s 
print N2s


datapath,avgs0=get_avg_array_for_Vs(V0s,0,day_dir_20160819,tag='OND_')
datapath,avgs1=get_avg_array_for_Vs(V1s,1,day_dir_20160819,tag='OND_')
datapath,avgs2=get_avg_array_for_Vs(V2s,2,day_dir_20160819,tag='OND_')

avgs = np.append(np.append(avgs1,avgs2,axis=0),avgs0,axis=0)
Ns = np.append(np.append(N1s,N2s),N0s)

#beunen begint hier
avgsN52=[(avgs[7]+avgs[8])/2.]
avgsN54=[(avgs[10]+avgs[11])/2.]
avgspos0 = np.append(np.append(np.append(np.append(avgs[0:7],avgsN52,axis=0),[avgs[9]],axis=0),avgsN54,axis=0),avgs[12:],axis=0)
print avgspos0
N52=[(Ns[7]+Ns[8])/2.]
N54=[(Ns[10]+Ns[11])/2.]
Nspos0 = np.append(np.append(np.append(np.append(Ns[0:7],N52),Ns[9]),N54),Ns[12:])
print Nspos0
#beunenn stopt hier

ax = plot_updown_lws(avgspos0,Nspos0)

ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L.png'))
plt.show()

plot_finesse_vs_length(avgs,Ns)


In [ ]:
# ON DIAMOND LINEWIDTH< LENGHTS< POS1 L2
day_dir_20160822 = 'D:\measuring\data/20160822'
V0s=np.array([1.546,3.019,4.237,5.348,6.404,7.473,8.515,9.551])
V3s=np.array([1.373,2.859,4.062,5.152,6.218,7.224,8.198,9.268])
V4s=np.array([0.951,2.390,3.646,4.783,5.843,6.934,7.975,8.977])
V5s=np.array([1.359,3.383,4.565,5.620,6.695,7.727,8.734,9.832])

N0s = np.arange(53,61,1)#55,63#53,61
N3s = np.arange(47,55,1)#49,57#46,55
N4s = np.arange(44,52,1)
N5s = np.arange(39,47,1)
print V3s
print N0s
print N3s
print N4s
print V0s
# 
datapath,avgs0=get_avg_array_for_Vs(V0s,1,day_dir_20160822,tag='OND_POS2_')
datapath,avgs3=get_avg_array_for_Vs(V3s,3,day_dir_20160822,tag='OND_POS2_')
datapath,avgs4=get_avg_array_for_Vs(V4s,4,day_dir_20160822,tag='OND_POS2_')
datapath,avgs5=get_avg_array_for_Vs(V5s,5,day_dir_20160822,tag='OND_POS2_')


avgs=np.append(np.append(np.append(avgs0,avgs3,axis=0),avgs4,axis=0),avgs5,axis=0)
Ns=np.append(np.append(np.append(N0s,N3s),N4s),N5s)
# avgs = np.append(np.append(avgs0,avgs1,axis=0),avgs2,axis=0)
# Ns = np.append(np.append(N0s,N1s),N2s)

ax = plot_updown_lws(avgs,Ns)
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L.png'))
plt.show()

plot_finesse_vs_length(avgs,Ns)
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'Finesse_vs_L.png'))
plt.show()


# ax = plot_mean_lws(avgs,Ns, label='mean pos 1')
# ax = plot_mean_lws(avgspos0,Nspos0, ax=ax, style ='go',label='mean pos 0')
# ax.set_title(datapath)
# plt.savefig(os.path.join(datapath,'plot_lw_vs_L_2positions.png'))
# plt.show()

In [ ]:
# ON DIAMOND LINEWIDTH< LENGHTS< POS0
#initial analayysis (manual)
linewidths1 = np.array([1.65,1.46, 1.62,1.70,1.67, 1.49,1.24,1.18])
linewidths2 = np.array([0.809,0.798,0.799,0.792])
linewidths3 = np.array([0.93,1.03,1.19])
u_linewidths1 = np.array([0.06,0.04,0.04,0.04,0.04,0.04,0.04,0.04])
u_linewidths2 = np.array([0.03,0.033,0.031,0.034])
u_linewidths3 = np.array([0.04,0.04,0.04])

linewidths=np.append(np.append(linewidths1 ,linewidths2),linewidths3)
u_linewidths=np.append(np.append(u_linewidths1 ,u_linewidths2),u_linewidths3)


In [ ]:
# OFF DIAMOND LINEWIDTH< LENGHTS< 

linewidths = np.array([1.26,1.17,1.09,1.04,0.97,0.92,0.88,0.83,0.79,0.77])*1.e9
u_linewidths = np.array([0.04,0.05,0.05,0.05,0.03,0.04,0.03,0.03,0.04,0.03])*1.e9
N = np.array([13,14,15,16,17,18,19,20,21,22])
cavity_lengths =  636.6*1.e-9*N/2
print linewidths,u_linewidths,cavity_lengths



In [ ]:
# V = 1.129
#8 um at  4.611, 
#so 
#removed 0.8+-0.03 @ N=22
#23,24,0.76,0.76,0.04,0.03,
fig,ax = plt.subplots()
ax.errorbar(cavity_lengths*1.e6,linewidths*1.e-9,yerr=u_linewidths*1.e-9,fmt='o')
ax.set_xlabel('cavity length (um)')
ax.set_ylabel('linewidth (GHz)')


In [ ]:
reload(common)
g_a = 0.1e9
g_b = (c)/(2*40000)
print g_b
print cavity_lengths,linewidths
p0, fitfunc, fitfunc_str = common.fit_inverse(g_a, g_b)
fit_result = fit.fit1d(cavity_lengths,linewidths, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=[])
print p0
fig,ax = plt.subplots(figsize=(8,4))
ax.plot(cavity_lengths,linewidths)
plot.plot_fit1d(fit_result, np.linspace(min(cavity_lengths),max(cavity_lengths),10*len(cavity_lengths)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='red', data_linestyle = 'o', print_info= True)
b_fit=fit_result['params_dict']['b']
Finesse = c/(2*b_fit)
ax.set_title(str(Finesse))
# ax.text(0.1,0.1,'the finesse is '+str(Finesse))
print Finesse

In [ ]:



def fit_all_resonances_in_sweep(nr_lws,x0,windowsize,x_odd_offset,g_gamma,
                                g_dx,g_A,max_chisq,test=False,total_length=12500,data_col=3):

    oa_single = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
    # oa_single.get_data(use_timetrace=False,x_min=x_min,x_max=x_max,data_col=3)#1400)
    # oa_single.plot_data(plot_mod=False,figsize=(16,4),fmt='o')
    #
    max_u_lw=0.1
    min_A2=0.1
    lws = np.array([])#lws_old## 
    As = np.array([])#lws_old## 
    gammas = np.array([])#lws_old## 
    chisqs= np.array([])
    x01s = np.array([])
    u_lws =np.array([])
    nr_lws=nr_lws
    x0=x0
    success =np.zeros(nr_lws)
    windowsize=windowsize# print len(x)

    
    for i in np.arange(nr_lws):
        if test:
            if i>10:
                break
        xi = int(x0+(total_length*i/nr_lws))
        print xi


        if i%2==1:
            xi=xi+x_odd_offset
        if (xi-windowsize)>0:
            x_min = xi-windowsize
        else: 
            x_min = 0
        if (xi+windowsize)<len(x)-1:
            x_max = xi+windowsize
        else:
            x_max = -1


        oa_single.get_data(use_timetrace=False,x_min = x_min, x_max=x_max,data_col=data_col)    
        oa_single.plot_data()

        lw,u_lw,A1,u_A1,gamma1,u_gamma1,chisq,x01,A2 = \
            oa_single.plot_and_fit_with_EOM(EOM_freq=EOM_freq,
               g_gamma1=g_gamma, g_dx=g_dx,g_A2=g_A,g_A3=g_A,show_fit=test,print_fit=test,save_fit=False)

        if chisq>0.4:
            print 20*'*'
            print chisq,lw
            print 'fit already not good enough! '
            print 20*'*'
            lws = np.append(lws,lw)
            As = np.append(As,A1)
            gammas = np.append(gammas,gamma1)
            chisqs = np.append(chisqs,chisq)
            x01s = np.append(x01s,x01)
            u_lws = np.append(u_lws,u_lw)

            continue

        if (x01-windowsize)>0:
            x_min = x01-windowsize    
        else: 
            x_min = 0
        if (x01+windowsize)<len(x)-1:
            x_max = x01+windowsize
        else:
            x_max = None
        oa_single.get_data(use_timetrace=False,x_min = x_min, x_max=x_max,data_col=data_col)
    #     oa_single.plot_data()    
        lw,u_lw,A1,u_A1,gamma1,u_gamma1,chisq,x02,A2 = \
            oa_single.plot_and_fit_with_EOM(EOM_freq=EOM_freq,
               g_gamma1=g_gamma, g_dx=g_dx,g_A2=g_A,g_A3=g_A,show_fit=True,print_fit=False,plot_name=str(int(round(x01,0))))

        u_lws = np.append(u_lws,u_lw)
        lws = np.append(lws,lw)
        As = np.append(As,A1)
        gammas = np.append(gammas,gamma1)
        chisqs = np.append(chisqs,chisq)
        x01s = np.append(x01s,x01)
        if chisq>max_chisq:
            print 20*'*'
            print chisq,lw,'+-',u_lw
            print 'fit not good enough! '
            print 20*'*'
            continue
        if u_lw > max_u_lw:
            print 20*'*'
            print chisq,lw,'+-',u_lw
            print 'fit not good enough! uncertainty in linewidth too large '
            print 20*'*'
            continue
        if A2 < min_A2:
            print 20*'*'
            print chisq,lw,'+-',u_lw
            print A2
            print 'fit not good enough! A2 is less than 0'
            print 20*'*'
            continue      

        print 20*'*'
        print chisq,lw,'+-',u_lw,'gamma',gamma1
        print 'fit good!!! '
        print 20*'*'
        success[i] = 1

    good_x01s = x01s[np.where(success>0)]
    good_lws = lws[np.where(success>0)]
    good_chisqs = chisqs[np.where(success>0)]
    good_u_lws = u_lws[np.where(success>0)]

    good_fits = {}
    good_fits['x0s'] =good_x01s
    good_fits['lws'] = good_lws
    good_fits['chisqs'] = good_chisqs
    good_fits['u_lws']=good_u_lws
    good_fits['max_chisq']=max_chisq
    good_fits['max_u_lw']=max_u_lw
    good_fits['min_A2']=min_A2

    print '%d good fits out of %d tries'%(len(good_lws),nr_lws)
    print 'the good linewidth fits from %s :'%filename
    print 'average lw = %.2f +- %.2f'%(np.average(good_lws),np.std(good_lws))
    print 'lws = ',good_lws
    print 'u_lws = ',good_u_lws
    print 'chisq = ',good_chisqs
    print 'x0s = ',good_x01s

    save_to_json_file(indir,filename+'_analysis',good_fits)
    return good_lws


In [ ]:
indir = 'D:\measuring\data/20160914/HR_ANN_PROC_L0/V-1'
filename = 'LWS006'
EOM_freq= 7.
data_col=2
reload(oscillo_ana)

oa_all = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
x,y=oa_all.get_data(use_timetrace=False,x_min=None,x_max=None,data_col=data_col)
oa_all.get_data(use_timetrace=False,x_min=None,x_max=100000,data_col=data_col)
oa_all.plot_data(plot_mod=False,figsize=(16,4),fmt='o')
print len(x)

test=True
x0 =2100
x_odd_offset =0

ms_per_div = 10 
mod_frequency = 50 #Hz

windowsize=200# print len(x)
g_gamma = 6
g_dx=26
g_A=1.4
max_chisq=0.01#0.003#0.007#0.004#0.001#0.04

nr_lws=20.*mod_frequency*ms_per_div/1000.



fit_all_resonances_in_sweep(nr_lws,x0,windowsize,x_odd_offset,g_gamma,g_dx,g_A,max_chisq,test=test,total_length=125000,data_col=data_col)



In [ ]:
#working, but rather gebeund cell to analyse a trace with linewidths at LT
indir = 'D:\measuring\data/20160909/LT_OND_POS4_L2/V0'
filename = 'LTLWS004'
EOM_freq= 6.
reload(oscillo_ana)

oa_all = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
x,y=oa_all.get_data(use_timetrace=False,x_min=None,x_max=None,data_col=3)
oa_all.get_data(use_timetrace=False,x_min=None,x_max=1500,data_col=3)
oa_all.plot_data(plot_mod=False,figsize=(16,4),fmt='o')

x_min=64000
x_max=x_min+5000
oa_single = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
# oa_single.get_data(use_timetrace=False,x_min=x_min,x_max=x_max,data_col=3)#1400)
# oa_single.plot_data(plot_mod=False,figsize=(16,4),fmt='o')
#

lws = np.array([])#lws_old## 
As = np.array([])#lws_old## 
gammas = np.array([])#lws_old## 
chisqs= np.array([])
x01s = np.array([])
u_lws =np.array([])
nr_lws=100
x0 =180
success =np.zeros(nr_lws)
windowsize=150# print len(x)

#for 10 ms per div, 1kHz sweep -> nr_lws = 100
#x0~200
#windowsize~150

print len(x)
print int(x0+(len(x)+399/nr_lws))

for i in np.arange(nr_lws):
#     if i>10:
#         break
    xi = int(x0+(len(x)*i/nr_lws))
    print xi


    if i%2==1:
        xi=xi-70
    if (xi-windowsize)>0:
        x_min = xi-windowsize
    else: 
        x_min = 0
    if (xi+windowsize)<len(x)-1:
        x_max = xi+windowsize
    else:
        x_max = -1
    

    oa_single.get_data(use_timetrace=False,x_min = x_min, x_max=x_max,data_col=3)    
    oa_single.plot_data()

    lw,u_lw,A1,u_A1,gamma1,u_gamma1,chisq,x01 = \
        oa_single.plot_and_fit_with_EOM(EOM_freq=EOM_freq,
           g_gamma1=10, g_dx=40,g_A2=0.6,g_A3=0.6,show_fit=True,print_fit=True)
        
    if chisq>0.2:
        print 20*'*'
        print chisq,lw
        print 'fit already not good enough! '
        print 20*'*'
        lws = np.append(lws,lw)
        As = np.append(As,A1)
        gammas = np.append(gammas,gamma1)
        chisqs = np.append(chisqs,chisq)
        x01s = np.append(x01s,x01)
        u_lws = np.append(u_lws,u_lw)

        continue
        
    if (x01-windowsize)>0:
        x_min = x01-windowsize    
    else: 
        x_min = 0
    if (x01+windowsize)<len(x)-1:
        x_max = x01+windowsize
    else:
        x_max = None
    oa_single.get_data(use_timetrace=False,x_min = x_min, x_max=x_max,data_col=3)
#     oa_single.plot_data()    
    lw,u_lw,A1,u_A1,gamma1,u_gamma1,chisq,x02 = \
        oa_single.plot_and_fit_with_EOM(EOM_freq=EOM_freq,
           g_gamma1=10, g_dx=40,g_A2=0.6,g_A3=0.6,show_fit=True,print_fit=False,plot_name=str(int(round(x01,0))))
        
    u_lws = np.append(u_lws,u_lw)
    lws = np.append(lws,lw)
    As = np.append(As,A1)
    gammas = np.append(gammas,gamma1)
    chisqs = np.append(chisqs,chisq)
    x01s = np.append(x01s,x01)
    if chisq>0.001:
        print 20*'*'
        print chisq,lw
        print 'fit not good enough! '
        print 20*'*'
        continue
    if u_lw > 0.1:
        print 20*'*'
        print chisq,lw,'+-',u_lw
        print 'fit not good enough! '
        print 20*'*'
        continue       
    
    print 20*'*'
    print chisq,lw
    print 'fit good!!! '
    print 20*'*'
    success[i] = 1
        
good_x01s = x01s[np.where(success>0)]
good_lws = lws[np.where(success>0)]
good_chisqs = chisqs[np.where(success>0)]
good_u_lws = u_lws[np.where(success>0)]

good_fits = {}
good_fits['x0s'] =good_x01s
good_fits['lws'] = good_lws
good_fits['chisqs'] = good_chisqs
good_fits['u_lws']=good_u_lws

print 'the good linewidth fits from %s :'%filename
print 'x0s = ',good_x01s
print 'lws = ',good_lws
print 'u_lws = ',good_u_lws
print 'chisq = ',good_chisqs

save_to_json_file(indir,filename+'_analysis',good_fits)

In [ ]:
reload(oscillo_ana)
indir='D:\measuring\data/20160825/OND_POS2L8'#'D:\measuring\data/20160430\OFF_diamond'
# filenames=['V0/NICE_LWS00168','V1/NICE_LWS00167','V-1/NICE_LWS00169',\
#           'V3/NICE_LWS00163','V4/NICE_LWS00164','V5/NICE_LWS00165',\
#           'V6/NICE_LWS00166']
indir = 'D:\measuring\data/20160830/135900'
filename = 'LT_LWS003'
filenames=[filename]#['V0/NICE_LWS00168','V1/NICE_LWS00167','V-1/NICE_LWS00169']
EOM_freq=6. # in GHz
# x0s = [200,170,850,650,1100,750,400]
# dx0s = [0,0,0,0,0,0,0,0,0]

x0s = [5000,700,850]
dx0s = [0,0,0]

for j,filename in enumerate(filenames):
    oa = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
    oa.get_data(use_timetrace=False,x_min=0,x_max=10000,data_col=3)#1400)
    oa.plot_data()#plot_mod=True)

    # lws_old=lws[:-1]
    lws = np.array([])#lws_old## 
    As = np.array([])#lws_old## 
    gammas = np.array([])#lws_old## 
    nr_lws=10

    x0 =x0s[j]
    windowsize=5000# print len(x)
    # xi = int(len(x)/nr_lws)
    # print xi 

    x,y = oa.get_data(use_timetrace=False,data_col=3)
    # lw,u_lw = oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq)
    # print lw

    for i in np.arange(nr_lws):
        xi = int(x0+(len(x)*i/nr_lws))

        if i%2==1:
            xi=xi+dx0s[j]   
        if (xi-windowsize)>0:
            x_min = xi-windowsize
        else: 
            x_min = 0
        if (xi+windowsize)<len(x)-1:
            x_max = xi+windowsize
        else:
            x_max =-1
        oa.get_data(use_timetrace=False,x_min = x_min, x_max=x_max,data_col=3)
    #     oa.plot_data()
        print i
        lw,u_lw,A1,u_A1,gamma1,u_gamma1 = \
            oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq,
                g_gamma1=500, g_dx=1000,g_A2=10000,g_A3=10000,show_fit=True,print_fit=False)
        lws = np.append(lws,lw)
        As = np.append(As,A1)
        gammas = np.append(gammas,gamma1)
        if i>1:
            break

    print lws
    print len(lws)
    print np.average(lws)
    print np.std(lws)

    f = open(os.path.join(indir,filename+'_fitted_lws.txt'), 'w')
    np.savetxt(f,lws)
    f.close()
    f = open(os.path.join(indir,filename+'_fitted_As.txt'), 'w')
    np.savetxt(f,As)
    f.close()
    f = open(os.path.join(indir,filename+'_fitted_gammas.txt'), 'w')
    np.savetxt(f,gammas)
    f.close()

# plt.show()

# x,y = oscillo_ana.plot_data(indir, filename,use_timetrace=False,x_min=x_min,x_max=x_max)

# oscillo_ana.plot_and_fit_with_EOM(indir, filename,EOM_frq,g_a1, g_A1, g_x01, g_gamma1, g_dx, g_A2,g_A3,fixed=fixed,use_timetrace=False)

In [ ]:

# print len(lws)
# print lws
# lws_old=lws[120:]
# print len(lws_old)
# print lws_old
# #print lws_old
# f = open(os.path.join(indir,'fitted_lws.txt'), 'w')
# np.savetxt(f,lws_old)
# f.close()

In [ ]:
# indir='D:\measuring\data/20160819/OND_L2_V9p866'#
lws_from_file={}
for filename in filenames:
    print filename

    f = open(os.path.join(indir,filename+'_fitted_lws.txt'), 'r')
    lws_from_file[filename] = np.loadtxt(f)
    f.close()
    
    print len(lws_from_file[filename])

    os.listdir(indir)
    # filename='NICE_LWS00058'
    reload(oscillo_ana)
    oa = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
    oa.get_data(use_timetrace=False,x_max=2000)#5000
    oa.plot_data(plot_mod=True)
    #oa.get_data(use_timetrace=False,x_min=-2000)
    #oa.plot_data(plot_mod=True)

In [ ]:
first_lw_directions = ['u','u','u','u','u','u','d']

In [ ]:
#IF SINGLE FILE EXECUTE THIS CELL:
for i,filename in enumerate(filenames):
    first_lw_direction = first_lw_directions[i]

    updown_mean_lws,lws_up,lws_down =get_up_down_lws(lws_from_file[filename],first_lw_direction)
    analysis=add_to_analysis_dict({},lws_up,lws_down,updown_mean_lws)
    save_to_json_file(indir,filename+'_analysis',analysis)

    print 'results of ',indir,filename
    print_results(analysis)


In [ ]:
#IF THREE FILES EXECUTE THIS AND THE FOLLWING CELLS:

file1_first_lw_direction = 'up'
file2_first_lw_direction = 'up'
file3_first_lw_direction = 'down'
removedfromfile1=0
removedfromfile2=0


In [ ]:
fitted_lws_file1=lws_from_file[0:48-removedfromfile1]
fitted_lws_file2=lws_from_file[48-removedfromfile1:96-removedfromfile1-removedfromfile2]
fitted_lws_file3=lws_from_file[96-removedfromfile1-removedfromfile2:144]

print len(fitted_lws_file1),len(fitted_lws_file2),len(fitted_lws_file3)
#print fitted_lws_file1
#print fitted_lws_file2 
#print fitted_lws_file3

In [ ]:
save_to_txt_file(indir,'fitted_lws_file1',fitted_lws_file1)
save_to_txt_file(indir,'fitted_lws_file2',fitted_lws_file2)
save_to_txt_file(indir,'fitted_lws_file3',fitted_lws_file3)

updown_mean_lws_file1,lws_up_file1,lws_down_file1 =get_up_down_lws(fitted_lws_file1,file1_first_lw_direction)
updown_mean_lws_file2,lws_up_file2,lws_down_file2 =get_up_down_lws(fitted_lws_file2,file2_first_lw_direction)
updown_mean_lws_file3,lws_up_file3,lws_down_file3 =get_up_down_lws(fitted_lws_file3,file3_first_lw_direction)

updown_mean_lws = np.append(np.append(updown_mean_lws_file1,updown_mean_lws_file2),updown_mean_lws_file3)
lws_up = np.append(np.append(lws_up_file1,lws_up_file2),lws_up_file3)
lws_down = np.append(np.append(lws_down_file1,lws_down_file2),lws_down_file3)

analysis=add_to_analysis_dict({},lws_up,lws_down,updown_mean_lws)
analysis_file1=add_to_analysis_dict({},lws_up_file1,lws_down_file1,updown_mean_lws_file1)
analysis_file2=add_to_analysis_dict({},lws_up_file2,lws_down_file2,updown_mean_lws_file2)
analysis_file3=add_to_analysis_dict({},lws_up_file3,lws_down_file3,updown_mean_lws_file3)
save_to_json_file(indir,'analysis',analysis)
save_to_json_file(indir,'analysis_file1',analysis_file1)
save_to_json_file(indir,'analysis_file2',analysis_file2)
save_to_json_file(indir,'analysis_file3',analysis_file3)

print 'results of ',indir
print_results(analysis)
print_results(analysis_file1)
print_results(analysis_file2)
print_results(analysis_file3)

In [ ]:
import matplotlib.pyplot as plt

fig,ax=plt.subplots()

ax.plot(lws)
ax.set_xlabel('sweep nr')
ax.set_ylabel('linewidth')

In [ ]:
f = open(os.path.join(indir,'fitted_lws_file0010.txt'), 'w')
np.savetxt(f,lws)
f.close()

In [ ]:
copy_lws_from_file=lws_from_file
print co

In [ ]:
f = open(os.path.join(indir,'fitted_lws.txt'), 'w')
np.savetxt(f,new_lws_from_file)
f.close()